In [2]:
# based on https://github.com/scipr-lab/ecfactory/

import time
from sage.all import is_prime, is_square, power_mod, fundamental_discriminant, log, floor
    
def is_valid_curve(q,t,r,k,D): 
    if q == 0 or t == 0 or r == 0 or k == 0 or D == 0:
        return False
    if not is_prime(q):
        return False 
    if not is_prime(r):
        return False
    if not fundamental_discriminant(D) == D:
        return False
    if D % 4 == 0: #check CM equation
        if not is_square(4*(t*t - 4*q)//D):
            return False
    if D % 4 == 1:
        if not is_square((t*t - 4*q)//D):
            return False
    if not (q+1-t) % r == 0: #check r | #E(F_q)
        return False
    if not power_mod(q,k,r) == 1: #check embedding degree is k
        return False
    return True

def filter_decorator(f):
    def helper(*args):
        q,t,r,k,D = f(*args)
        num_bits = _number_of_bits(r)
        while not is_suitable_curve(q,t,r,k,D, num_bits):
            q,t,r,k,D = f(*args)
            num_bits = _number_of_bits(r)
        return q,t,r,k,D
    return helper
    
def _number_of_bits(n):   
    if n == 0:
        return 1
    else:
        return floor(log(n).n()/log(2).n()) + 1

def is_suitable_curve(q,t,r,k,D, num_bits):
    return _number_of_bits(r) >= num_bits and is_valid_curve(q,t,r,k,D)

def is_suitable_q(q):
    return is_prime(q)

def is_suitable_r(r):
    return is_prime(r)
    
    
def print_curve(q,t,r,k,D):
    print(curve_to_string(q,t,k,r,D))
    
def curve_to_string(q,t,k,r,D):
    if q == 0 or t == 0 or r == 0 or k == 0 or D == 0:
        return 'Failed to find an elliptic curve'
    else:
        return 'Elliptic curve over a field of size ' + str(q) + ' with trace ' + str(t) + ', a subgroup of order ' + str(r) + ' with embedding degree ' + str(k) + ', and fundamental discriminant ' + str(D)

from sage.all import random, power_mod, primitive_root, Integer, random_prime, is_prime, kronecker, squarefree_part, is_square, Mod, fundamental_discriminant, randint
import time

def find_element_of_order(k,r):
    assert r % k == 1
    h = 0
    def order(h,k,p):
        bool = True
        g = h
        for i in range(1,k):
            bool = bool and g != 1
            g = Mod(g * h, p)
        bool = bool and g == 1
        return bool
    while not order(h,k,r): # expected number of iterations is k/euler_phi(k)
        h = power_mod(randint(2, r-1), (r-1)//k, r)
    return h


def method(r,k,D,max_trials=10000, g=0): 
    assert test_promise(r,k,D), 'Invalid inputs'
    if g != 0:
        assert power_mod(g,k,r) == 1, 'Invalid inputs'
    else:
        g = find_element_of_order(k,r)
    D = Integer(D)
    t = Integer(g) + 1
    root_d = Integer(Mod(D, r).sqrt())
    u = Integer(Mod((t-2)*root_d.inverse_mod(r) ,r))
    q = 1
    j = Integer(0)
    i = Integer(pow(2,127))
    count = 0
    while (count < max_trials):
           q = Integer( (t+i*r)**2 - D*(u + j*r)**2)
           if q % 4 ==0:
                q = q//4
                if is_suitable_q(q):
                    return (q, t+i*r)
                q = 1
           if random() < 0.5:
                j+=1
           else:
                i+=1
           count+=1
    return (0, 0) # no prime found, so end

@filter_decorator
def run(r,k,D,max_run_time=20):
    assert test_promise(r,k,D), 'Invalid inputs'
    q = 0
    t = 0
    start = time.time()
    while q == 0 and time.time() - start < max_run_time: # run cocks pinch method until successful
        q,t = method(r,k,D)
    assert is_valid_curve(q,t,r,k,D), 'Invalid output'
    return q,t,r,k,D

def gen_params_from_bits(num_bits,k): 
    r = random_prime(2**num_bits, lbound=2**(num_bits-1))
    while not (r % k == 1 and is_suitable_r(r)):
        r = random_prime(2**num_bits, lbound=2**(num_bits-1))
    return gen_params_from_r(r,k)

def gen_params_from_r(r,k):
    D = -Integer(Mod(int(random()*(1000)),r))
    i = 0
    while not kronecker(D,r) == 1: # expected number of iterations of the while loop is 2
        D = -Integer(Mod(int(random()*(1000)),r))
        i+=1
    D = fundamental_discriminant(D)
    if not (kronecker(D,r) == 1):
        return r, k, 0
    return r,k,D


def test_promise(r,k,D):
    bool = (kronecker(D,r) == 1) # D is a square mod r
    bool = bool and ( (r-1) % k ==0) # k | r-1
    bool = bool and (D == fundamental_discriminant(D)) # check that D is a fundamental discriminant
    return bool

In [4]:
all_curves = []

k = 6
r = 115792089237316195423570985008687907853269984665640564039457584007908834671663
D = -3

from sage.all import ln
trials = Integer((ln(r) * ln(2)).round())

print("Plan to sample " + str(trials) + " curves")

for i in range(trials):
    q,t,r,k,D = run(r,k,D)
    all_curves.append((q, t))
    print("Curve " + str(i + 1) + " found")

Plan to sample 123 curves
Curve 1 found
Curve 2 found
Curve 3 found
Curve 4 found
Curve 5 found
Curve 6 found
Curve 7 found
Curve 8 found
Curve 9 found
Curve 10 found
Curve 11 found
Curve 12 found
Curve 13 found
Curve 14 found
Curve 15 found
Curve 16 found
Curve 17 found
Curve 18 found
Curve 19 found
Curve 20 found
Curve 21 found
Curve 22 found
Curve 23 found
Curve 24 found
Curve 25 found
Curve 26 found
Curve 27 found
Curve 28 found
Curve 29 found
Curve 30 found
Curve 31 found
Curve 32 found
Curve 33 found
Curve 34 found
Curve 35 found
Curve 36 found
Curve 37 found
Curve 38 found
Curve 39 found
Curve 40 found
Curve 41 found
Curve 42 found
Curve 43 found
Curve 44 found
Curve 45 found
Curve 46 found
Curve 47 found
Curve 48 found
Curve 49 found
Curve 50 found
Curve 51 found
Curve 52 found
Curve 53 found
Curve 54 found
Curve 55 found
Curve 56 found
Curve 57 found
Curve 58 found
Curve 59 found
Curve 60 found
Curve 61 found
Curve 62 found
Curve 63 found
Curve 64 found
Curve 65 found
Curve 66

In [9]:
import multiprocess
import time

counter = 0
for x in all_curves: 
    counter += 1
    def worker(qu, number):
        from sage.all import factor 
        factor(number)
        qu.put(1)
        return 0
    
    qu = multiprocess.Queue()
    v = x[0] - 1
    th = multiprocess.Process(target = worker, args = (qu, v, ))
    th.start()
    print("Curve " + str(counter) + " is being tested.")
    time.sleep(15)
    if qu.empty():
        print("Curve " + str(counter) + " cannot be factored within a short amount of time.")
        th.terminate()
    else:
        print("q = " + str(x[0]))
        print("t = " + str(x[1]))
        q = x[0]
        t = x[1]
        from sage.all import factor 
        print("factor(q - 1):")
        print(factor(v))
        th.join()
        break

Curve 1 is being tested.
Curve 1 cannot be factored within a short amount of time.
Curve 2 is being tested.
Curve 2 cannot be factored within a short amount of time.
Curve 3 is being tested.
Curve 3 cannot be factored within a short amount of time.
Curve 4 is being tested.
Curve 4 cannot be factored within a short amount of time.
Curve 5 is being tested.
Curve 5 cannot be factored within a short amount of time.
Curve 6 is being tested.
Curve 6 cannot be factored within a short amount of time.
Curve 7 is being tested.
Curve 7 cannot be factored within a short amount of time.
Curve 8 is being tested.
Curve 8 cannot be factored within a short amount of time.
Curve 9 is being tested.
Curve 9 cannot be factored within a short amount of time.
Curve 10 is being tested.
Curve 10 cannot be factored within a short amount of time.
Curve 11 is being tested.
q = 970323807687943084468112180289064107569315242869752796872292783067802059198630185584372148002488819414213809737007869919470226655125798206

In [19]:
from sage.all import power_mod, primitive_root, is_square, hilbert_class_polynomial, is_prime,GF, EllipticCurve, kronecker, randint, Integer, Mod

In [20]:
def make_curve(q,t,r,k,D,debug=False):
    assert is_valid_curve(q,t,r,k,D), 'Invalid input. No curve exists.' # check inputs
    if debug:
        print('Tested input')
    poly = hilbert_class_polynomial(D) # compute hilbert class polynomial
    if debug:
        print('Computed Hilbert class polynomial')
    check = False
    j_inv = poly.any_root(GF(q)) # find j-invariant    
    orig_curve = EllipticCurve(GF(q), j=j_inv) # make a curve
    E = orig_curve
    check = test_curve(q,t,r,k,D,E) # see if this is the right curve
    twist = False
    if not check: # not the right curve, use quadratic twist
        E = E.quadratic_twist()
        check = test_curve(q,t,r,k,D,E)
        if check:
            twist = True
        else: # twist didnt work => j = 0 or 1728
            if j_inv == 0: # for j = 0, use sextic twists
                prim = primitive_root(q)
                i = 1
                while t != E.trace_of_frobenius() and i < 6:
                    E = orig_curve.sextic_twist(power_mod(prim,i,q))
                    i+=1
            elif j_inv == 1728: # for j = 1728, use quartic twists
                prim = primitive_root(q)
                i = 1
                while t != E.trace_of_frobenius() and i < 4:
                    E = orig_curve.quartic_twist(power_mod(prim,i,q))
                    i+=1
            else: # twist didnt work and j != 0, 1728. this should never happen, so write input to a file for debugging
                print('Error. Quadratic twist failed to find the correct curve with j != 0, 1728. Logging output to debug.txt') # this line should never be reached'
                f = open('debug.txt', 'w')
                f.write('Twist: ' + str(twist) + '\n')
                f.write('q: ' + str(q) + '\n')
                f.write('t: ' + str(t) + '\n')
                f.write('r: ' + str(r) + '\n')
                f.write('k: ' + str(k) + '\n')
                f.write('D: ' + str(D) + '\n')
                f.write('E: ' + str(E) + '\n')
                f.write('orig_curve: ' + str(orig_curve))
                f.close()
                return False
            check = test_curve(q,t,r,k,D,E)
            twist = True
    if not check: # didnt find a curve. this should never happen, so write input to a file for debugging
        print('Error. Failed to find curve. Logging output to debug.txt')
        f = open('debug.txt', 'w')
        f.write('Twist: ' + str(twist) + '\n')
        f.write('q: ' + str(q) + '\n')
        f.write('t: ' + str(t) + '\n')
        f.write('r: ' + str(r) + '\n')
        f.write('k: ' + str(k) + '\n')
        f.write('D: ' + str(D) + '\n')
        f.write('E: ' + str(E) + '\n')
        f.write('orig_curve: ' + str(orig_curve))
        f.close()
        return False
    return E
    
def small_B_twist(E):
    b = E.ainvs()[4]
    q = E.base_field().order()
    b = power_mod(Integer(b), -1, q)
    d = 0
    s = Mod(1,q)
    bool = True
    while bool:
        try:
            d = (s*b)
            d = d.nth_root(3)
            d = Integer(d)
            bool = False
        except ValueError as e:
            s+=1 
            pass
    ainvs = [i for i in E.ainvs()]
    ainvs[3] *= d**2
    ainvs[4] *= d**3
    return EllipticCurve(E.base_field(), ainvs)

def test_curve(q,t,r,k,D,E): 
    bool = True
    bool = bool and (power_mod(q, k, r) == 1) #q^k -1 ==0 mod r
    bool = bool and (E.trace_of_frobenius() == t)
    bool = bool and (kronecker((t*t-4*q) * Integer(D).inverse_mod(q),q) == 1)
    bool = bool and (E.cardinality() == q+1-t)
    bool = bool and (E.cardinality() % r ==0)
    return bool

In [21]:
E=make_curve(q,t,r,k,D)

In [22]:
small_B_twist(E)

Elliptic Curve defined by y^2 = x^3 + 2 over Finite Field of size 97032380768794308446811218028906410756931524286975279687229278306780205919863018558437214800248881941421380973700786991947022665512579820646198961959098333512765195892419457571665269791440752474042097029362343188600995015738227039